In [3]:
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.optim import Adam
import matplotlib.pyplot as plt

import lie_tools
from lie_tools import rodrigues

%config IPCompleter.greedy=True

In [4]:
def get_pointcloud(k):
    x = torch.tensor(np.random.normal(0, 1, (k, 3)), dtype=torch.float32)
    return x

In [5]:
def get_se3(so3, r):
    fill_shape = list(r.shape[:-2])
    filler = torch.tensor([[0.,0.,0.,1.]]).view([1]*\
                            (len(fill_shape)+1)+[4]).repeat(fill_shape + [1,1])
    se3 = torch.cat([so3, r], -1).type(torch.float32)
    se3 = torch.cat([se3, filler], -2)
    return se3

def do_se3_action(se3, x):
    # [tuple1, 4, 4]
    # x = [tuple2, 3]
    # x_hat = [tuple2, tuple1, 3]
    
    tuple1 = list(se3.shape[:-2])
    tuple2 = list(x.shape[:-1])
    
    ones1 = [1]*len(tuple1)
    ones2 = [1]*len(tuple2)
    
    se3 = se3.view(tuple1 + ones2 + [4,4])
    x_hat = torch.cat([x, torch.ones(tuple2+[1])], -1)
    x_hat = se3 @ x_hat.view(ones1 + tuple2 + [4,1])
    
    return x_hat[...,[0,1,2],0]
    
do_se3_action(get_se3(so3, r), get_pointcloud(7))

tensor([[-1.1379, -0.4028, -0.4629],
        [ 0.0874,  1.0766, -0.6072],
        [-1.7226, -0.9780, -1.5228],
        [-1.2961,  1.5090, -1.9034],
        [-0.6937,  0.2916,  0.6668],
        [-1.6789,  2.1931,  0.2377],
        [-2.0712, -1.3042,  0.9699]])

In [8]:
N_POINTS = 3
# create dataset 

# generate random SE3 element 
v = torch.tensor(np.random.normal(0, 1, (3)))
so3 = rodrigues(v)
r = torch.tensor(np.random.normal(0, 1, (3,1)))

se3_true = get_se3(so3, r)

# generate point cloud 
cloud = get_pointcloud(N_POINTS)

rotated_cloud = do_se3_action(se3_true, cloud)

In [ ]:
N_ITER = 100
N_SAMPLES = 2



